In [0]:
%python
import pandas as pd
import requests
import json

loading in the necessary columns using python pandas

In [0]:
%python
response = requests.get('https://api.tfl.gov.uk/Line/Mode/tube/Status').json()
df = pd.json_normalize(response, 'lineStatuses', ['name', 'modified'], record_prefix='_', errors='ignore')
df = df[['modified', 'name', '_statusSeverityDescription', '_reason']]
df.rename(columns={'modified': 'current_timestamp', 'name': 'line', '_statusSeverityDescription': 'status', '_reason': 'disruption_reason' }, inplace=True)

In [0]:
%python
df

,current_timestamp,line,status,disruption_reason
0,2024-05-14T14:35:06.647Z,Bakerloo,Good Service,NaN
1,2024-05-14T14:35:06.63Z,Central,Minor Delays,Central Line: Minor delays between Epping and ...
2,2024-05-14T14:35:06.647Z,Circle,Good Service,NaN
3,2024-05-14T14:35:06.63Z,District,Good Service,NaN
4,2024-05-14T14:35:06.647Z,Hammersmith & City,Good Service,NaN
5,2024-05-14T14:35:06.63Z,Jubilee,Good Service,NaN
6,2024-05-14T14:35:06.647Z,Metropolitan,Good Service,NaN
7,2024-05-14T14:35:06.647Z,Northern,Good Service,NaN
8,2024-05-14T14:35:06.63Z,Piccadilly,Good Service,NaN
9,2024-05-14T14:35:06.617Z,Victoria,Minor Delays,Victoria Line: Minor delays while we fix a poi...


loading the dataframe into a SQL table

In [0]:
%python
spark_df = spark.createDataFrame(df)

In [0]:
%python
spark_df.write.mode("overwrite").saveAsTable("tfl_status.status")

In [0]:
%python
spark.sql('select * from tfl_status.status')

Out[6]: DataFrame[current_timestamp: string, line: string, status: string, disruption_reason: string]

In [0]:
%sql
CREATE DATABASE tfl_status;

In [0]:
%sql
CREATE TABLE status (
	current_timestamp varchar(50),
	line varchar(20),
  status varchar(20),
  disruption_reason varchar(255)
);

In [0]:
%sql
select * from tfl_status.status;

current_timestamp,line,status,disruption_reason
2024-05-14T14:35:06.647Z,Bakerloo,Good Service,null
2024-05-14T14:35:06.63Z,Central,Minor Delays,Central Line: Minor delays between Epping and Leytonstone due to an earlier trespasser on the track at Leytonstone. GOOD SERVICE on the rest of the line.
2024-05-14T14:35:06.647Z,Circle,Good Service,null
2024-05-14T14:35:06.63Z,District,Good Service,null
2024-05-14T14:35:06.647Z,Hammersmith & City,Good Service,null
2024-05-14T14:35:06.63Z,Jubilee,Good Service,null
2024-05-14T14:35:06.647Z,Metropolitan,Good Service,null
2024-05-14T14:35:06.647Z,Northern,Good Service,null
2024-05-14T14:35:06.63Z,Piccadilly,Good Service,null
2024-05-14T14:35:06.617Z,Victoria,Minor Delays,Victoria Line: Minor delays while we fix a points failure at Brixton.
